In [1]:
import sys 
from pathlib import Path
src_dir = Path.cwd().parent
sys.path.append(str(src_dir))

from models.LP import LabelPropagation , LabelSpreading
from utilits.utilits import *
from models.Classifier_model import ClassifierModel
from models.simCSE import SimCSE

## load Datasets

In [3]:
olid_dataset = prepare_data("data/")
olid_dataset

{'train': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 13240
 }),
 'test': Dataset({
     features: ['tweets', 'labels', 'labels_name'],
     num_rows: 3887
 })}

In [4]:
#split data to train_dev 
training_data_m1 = split_data(olid_dataset["train"] , annotated_data_prec=0.8)
training_data_m1

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
})

### Train Classifier Model on all oiled_Dataset 

In [5]:
m1 = ClassifierModel()
m1.train(training_data_m1["train"],
         training_data_m1["test"] ,
         num_epochs=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.499200,0.434497,0.801360,0.712941,0.682432,0.800249
2,0.396700,0.435472,0.804003,0.720958,0.677928,0.802442


In [6]:
m1_metrics = m1.test(olid_dataset["test"])
m1_metrics

  0%|          | 0/1 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweets, labels_name. If tweets, labels_name are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'test_loss': 0.20792534947395325,
 'test_accuracy': 0.9264214046822743,
 'test_precision': 0.7989457831325302,
 'test_recall': 0.9824074074074074,
 'test_f1': 0.9285100475937289,
 'test_runtime': 4.3083,
 'test_samples_per_second': 902.205,
 'test_steps_per_second': 112.805}

In [9]:
all_m_metrics = dict()
all_m_metrics["All_dataset"] = m1_metrics

## annotate ulabeled data with self-supervised learning with LP 

In [12]:
# 10% of data labeld
annoteted_data = split_data(olid_dataset["train"] ,annotated_data_prec = 0.1)
annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 1324
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 11916
    })
})

In [13]:
from Label_Data import DataLabeling

In [14]:
LD = DataLabeling()
preds_labels =LD.generate_labels(annoteted_data["train"]["tweets"],
                                 annoteted_data["train"]["labels"],
                                 annoteted_data["test"]["tweets"])

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1480925a23f7db13cea1c830922dbd4173c2a1ccab8c57cbb36a1ea693164879.01dc297b74ef2153586ff6f1113a3309f339a11f1cef9d887ae2314924e8d17e
Model config RobertaConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

In [15]:
training_annoteted_data = create_dataset_of_label_propagation(annoteted_data["test"]["tweets"] ,
                                                             preds_labels)
training_annoteted_data = split_data(training_annoteted_data , annotated_data_prec = 0.8)
training_annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 9532
    })
    test: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 2384
    })
})

In [16]:
m3 = ClassifierModel()
m3.train(training_annoteted_data["train"],
         training_annoteted_data["test"] ,
         num_epochs=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.465400,0.413597,0.804950,0.629726,0.727410,0.808959
2,0.336100,0.410236,0.822987,0.758547,0.534639,0.812433
3,0.278000,0.413283,0.822148,0.720588,0.590361,0.816311


In [17]:
m2_metrics = m3.test(olid_dataset["test"])

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweets, labels_name. If tweets, labels_name are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


In [19]:
all_m_metrics["10%_labeled_data"] = m2_metrics
all_m_metrics

{'All_dataset': {'test_loss': 0.20792534947395325,
  'test_accuracy': 0.9264214046822743,
  'test_precision': 0.7989457831325302,
  'test_recall': 0.9824074074074074,
  'test_f1': 0.9285100475937289,
  'test_runtime': 4.3083,
  'test_samples_per_second': 902.205,
  'test_steps_per_second': 112.805},
 '10%_labeled_data': {'test_loss': 0.3510426878929138,
  'test_accuracy': 0.8487265243118086,
  'test_precision': 0.7557172557172557,
  'test_recall': 0.6731481481481482,
  'test_f1': 0.8459128148968845,
  'test_runtime': 4.2167,
  'test_samples_per_second': 921.811,
  'test_steps_per_second': 115.256}}

# with 20% annoteted

In [20]:
annoteted_data = split_data(olid_dataset["train"] , annotated_data_prec= 0.2)
annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 2648
    })
    test: Dataset({
        features: ['tweets', 'labels', 'labels_name'],
        num_rows: 10592
    })
})

In [21]:
LD = DataLabeling()
preds_labels =LD.generate_labels(annoteted_data["train"]["tweets"],
                                 annoteted_data["train"]["labels"],
                                 annoteted_data["test"]["tweets"])

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1480925a23f7db13cea1c830922dbd4173c2a1ccab8c57cbb36a1ea693164879.01dc297b74ef2153586ff6f1113a3309f339a11f1cef9d887ae2314924e8d17e
Model config RobertaConfig {
  "_name_or_path": "princeton-nlp/sup-simcse-roberta-base",
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "v

In [22]:
training_annoteted_data = create_dataset_of_label_propagation(annoteted_data["test"]["tweets"] ,
                                                             preds_labels)
training_annoteted_data = split_data(training_annoteted_data , annotated_data_prec = 0.8)
training_annoteted_data

DatasetDict({
    train: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 8473
    })
    test: Dataset({
        features: ['tweets', 'labels'],
        num_rows: 2119
    })
})

In [23]:
m4 = ClassifierModel()
m4.train(training_annoteted_data["train"],
         training_annoteted_data["test"] ,
         num_epochs=3)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.3",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.474100,0.420071,0.809344,0.808581,0.414552,0.786844
2,0.336100,0.369622,0.835300,0.730916,0.648054,0.832118
3,0.278300,0.381652,0.836715,0.716049,0.686971,0.835660


In [26]:
m3_metrics = m4.test(olid_dataset["test"])
all_m_metrics["20%_labeled_data"] = m3_metrics

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: tweets, labels_name. If tweets, labels_name are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3887
  Batch size = 8


In [28]:
all_m_metrics

{'All_dataset': {'test_loss': 0.20792534947395325,
  'test_accuracy': 0.9264214046822743,
  'test_precision': 0.7989457831325302,
  'test_recall': 0.9824074074074074,
  'test_f1': 0.9285100475937289,
  'test_runtime': 4.3083,
  'test_samples_per_second': 902.205,
  'test_steps_per_second': 112.805},
 '10%_labeled_data': {'test_loss': 0.3510426878929138,
  'test_accuracy': 0.8487265243118086,
  'test_precision': 0.7557172557172557,
  'test_recall': 0.6731481481481482,
  'test_f1': 0.8459128148968845,
  'test_runtime': 4.2167,
  'test_samples_per_second': 921.811,
  'test_steps_per_second': 115.256},
 '20%_labeled_data': {'test_loss': 0.32330816984176636,
  'test_accuracy': 0.8597890403910471,
  'test_precision': 0.7407740774077408,
  'test_recall': 0.7620370370370371,
  'test_f1': 0.8603916825329474,
  'test_runtime': 4.2219,
  'test_samples_per_second': 920.667,
  'test_steps_per_second': 115.113}}